# OpenAI Agent with LlamaIndex

## Install Dependencies

In [1]:
! pip install uv
! uv pip install --system -q python-dotenv llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai-like "openinference-instrumentation-llama-index>=2" "arize-phoenix>=4.29.0" python-dotenv

## Setup API Keys
To run the rest of the notebook you will need access to an OctoAI API key. You can sign up for an account [here](https://octoai.cloud/). If you need further guidance you can check OctoAI's [documentation page](https://octo.ai/docs/getting-started/how-to-create-octoai-access-token).

In [4]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]

## Import libraries and setup LlamaIndex

In [5]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

# Set up Phoenix

In [6]:
import phoenix as px
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [7]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

OpenTelemetry Tracing Details
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Load Documents

In [8]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/tokyo.txt"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/seattle.txt"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

This is the point we create our vector indexes, by calculating the embedding vectors for each of the chunks. You only need to run this once.

In [9]:
if not index_loaded:
    # load data
    tokyo_docs = SimpleDirectoryReader(
        input_files=["./city_data/tokyo.txt"]
    ).load_data()
    seattle_docs = SimpleDirectoryReader(
        input_files=["./city+data/seattle.txt"]
    ).load_data()

    # build index
    tokyo_index = VectorStoreIndex.from_documents(tokyo_docs, show_progress=True)
    seattle_index = VectorStoreIndex.from_documents(seattle_docs, show_progress=True)

    # persist index
    # lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    # uber_index.storage_context.persist(persist_dir="./storage/uber")

Parsing nodes:   0%|          | 0/238 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/343 [00:00<?, ?it/s]

Now create the query engines.

In [10]:
tokyo_engine = lyft_index.as_query_engine(similarity_top_k=3, llm=llm)
seattle_engine = uber_index.as_query_engine(similarity_top_k=3, llm=llm)

We can now define the query engines as tools that will be used by the agent.

As there is a query engine per document we need to also define one tool for each of them.

In [11]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=tokyo_engine,
        metadata=ToolMetadata(
            name="tokyo_info",
            description=(
                "Provides information about the history of Tokyo."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=seattle_engine,
        metadata=ToolMetadata(
            name="seattle_info",
            description=(
                "Provides information about the history of Seattle. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Creating the Agent
Now we have all the elements to create a LlamaIndex ReactAgent

In [12]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

Now we can interact with the agent and ask a question.

In [18]:
response = agent.chat("How far away is london?")
print(str(response))

> Running step b694d7e7-c94e-4d4b-8059-b2da878488ec. Step input: How far away is london?
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 67cc3db5-4947-4cc1-a95a-e8f74f3b4a43. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 2ffa5bbc-fd2c-43c0-83fd-58daf2c0b264. Step input: None
Thought: The user's question is incomplete. I need more information to provide a precise answer. The distance to London would depend on the user's current location, which has not been specified.
Answer: To provide the distance to London, I would need to know your current location. Could you please provide that information?
To provide the distance to London, I would need to know your current location. Could you please provide that information?
